# TP53 Mutation Trans Effect on Phosphoproteomics

### Library Imports

In [1]:
import pandas as pd
import numpy as np
import scipy.stats

import warnings
warnings.filterwarnings("ignore")

import cptac
import cptac.utils as al

brain = cptac.Gbm()

### Select Gene

In [2]:
gene = "TP53"

### Investigate Proteomics, Phosphoproteomics, Acetylproteomics, or Transcriptomics

In [3]:
#omics = "proteomics"
#omics = "transcriptomics"
omics = "phosphoproteomics"
#omics = "acetylproteomics"

### Track all significant comparisons in Dataframe

In [4]:
all_significant_comparisons = pd.DataFrame(columns=['Cancer_Type', 'Gene', 'Comparison','Interacting_Protein','P_Value'])

In [5]:
def add_to_all_significant_comparisons(df, cancer, interacting, all_sig_comp):
    expanded = df
    expanded['Gene'] = gene
    expanded['Cancer_Type'] = cancer
    expanded['Interacting_Protein'] = interacting
    
    updated_all_comparisons = pd.concat([all_sig_comp, expanded], sort=False)
    
    return updated_all_comparisons

# Interacting Proteins: Phosphoproteomics

### Generate interacting protein list

Make a call to cptac.utils to get the interacting proteins method, which interacts with the uniprot and string databases to generate a list of known interacting partners with the given gene.

In [6]:
# Use get interacting proteins method to generate list of interacting proteins
interacting_proteins = al.get_interacting_proteins(gene)

print("Interacting Proteins:")
for interacting_protein in interacting_proteins:
    print(interacting_protein)

Interacting Proteins:
RPA1
AURKA
MDM2
SIRT1
CREBBP
EP300
CDK2
TP53
ATM
BCL2L1
TP53BP2
CCNG1
MDM4
PTEN
TP53BP1
CHEK2
FZR1
MAPK8
KDM1A
BCL6
CDKN1A
HIPK2
CHEK1
CDKN2A
BRCA1
AKT1
DVL2
BLM
ATR
TFAP2A
EPHA3


### Test for significant comparisons in any of interacting proteins

In [7]:
# Create dataframe in order to do comparisons with wrap_ttest
protdf = brain.join_omics_to_mutations(mutations_genes=[gene], omics_df_name=omics, omics_genes=interacting_proteins)
protdf = brain.reduce_multiindex(protdf, levels_to_drop=1, flatten = True)
protdf = protdf.loc[protdf['Sample_Status'] == 'Tumor']

# Create the binary valued column needed to do the comparison
for ind, row in protdf.iterrows():
    if row[gene+"_Mutation_Status"] != 'Wildtype_Tumor':
        protdf.at[ind,'Label'] = 'Mutated'
    else:
        protdf.at[ind,'Label'] = 'Wildtype'

# Format the dataframe correctly'''
protdf = protdf.drop(gene+"_Mutation",axis=1)
protdf = protdf.drop(gene+"_Location",axis=1)
protdf = protdf.drop(gene+"_Mutation_Status", axis=1)
protdf = protdf.drop("Sample_Status",axis=1)


# Make list of columns to be compared using t-tests
col_list = list(protdf.columns)
col_list.remove('Label')

print("Doing t-test comparisons\n")

# Call wrap_ttest, pass in formatted dataframe
wrap_results = al.wrap_ttest(protdf, 'Label', col_list)

# Print results, if anything significant was found
if wrap_results is not None:
        print(wrap_results)
        print("\n\n")
        
        all_significant_comparisons = add_to_all_significant_comparisons(wrap_results, "Gbm", True, all_significant_comparisons)


Doing t-test comparisons

                                          Comparison       P_Value
0  TP53_phosphoproteomics_ALPNNTSSS*PQPK_NP_000537.3  1.539204e-07
1  TP53BP1_phosphoproteomics_QSQQPMKPIS*PVKDPVS*P...  1.351309e-06
2  TP53BP1_phosphoproteomics_S*PEPEVLST*QEDLFDQSN...  2.371163e-06
3  TP53BP1_phosphoproteomics_GNLLHFPSS*QGEEEK_NP_...  1.266342e-05
4  TP53BP1_phosphoproteomics_SGTAETEPVEQDSS*QPSLP...  7.026147e-05
5  TP53BP1_phosphoproteomics_EGDIIPPLTGAT*PPLIGHL...  7.413043e-05
6  TP53BP1_phosphoproteomics_DPVS*PAS*QK_NP_00113...  9.519195e-05





## All Proteins: Phosphoproteomics

In [8]:
try:
    print("\nGene: ", gene)

    # Use all proteins
    proteomics = brain.get_proteomics()
    all_proteins = proteomics.columns
    #all_proteins = all_proteins[:100]

    # Create dataframe in order to do comparisons with wrap_ttest
    protdf = brain.join_omics_to_mutations(mutations_genes=[gene], omics_df_name=omics)
    protdf = brain.reduce_multiindex(protdf, flatten = True)
    protdf = protdf.loc[protdf['Sample_Status'] == 'Tumor']
    
    # Create the binary valued column needed to do the comparison
    for ind, row in protdf.iterrows():
        if row[gene+"_Mutation_Status"] != 'Wildtype_Tumor':
            protdf.at[ind,'Label'] = 'Mutated'
        else:
            protdf.at[ind,'Label'] = 'Wildtype'

    # Format the datafram correctly'''
    protdf = protdf.drop(gene+"_Mutation",axis=1)
    protdf = protdf.drop(gene+"_Location",axis=1)
    protdf = protdf.drop(gene+"_Mutation_Status", axis=1)
    protdf = protdf.drop("Sample_Status",axis=1)

    # Make list of columns to be compared using t-tests
    col_list = list(protdf.columns)
    col_list.remove('Label')

    print("Doing t-test comparisons\n")
    
    # Call wrap_ttest, pass in formatted dataframe
    wrap_results = al.wrap_ttest(protdf, 'Label', col_list)

    # Print results, if anything significant was found
    if wrap_results is not None:
            print(wrap_results)
            print("\n\n")
            
            all_significant_comparisons = add_to_all_significant_comparisons(wrap_results, "Gbm", False, all_significant_comparisons)


except Exception as e:
    print("Error in Comparison")
    print(e)


Gene:  TP53


Doing t-test comparisons

                                          Comparison       P_Value
0  TCEAL3_phosphoproteomics_S65_REDEGEPGDEGQLEDEG...  1.559019e-09
1  TCEAL3_phosphoproteomics_S72_QGRS*EGEGKPQGEGK_...  3.528867e-08
2  TP53_phosphoproteomics_S315_ALPNNTSSS*PQPK_NP_...  1.539204e-07





### Print all significant comparisons

In [9]:
if len(all_significant_comparisons) > 0:
    display(all_significant_comparisons)
    
else:
    print('No Significant Comparisons!')

,Cancer_Type,Gene,Comparison,Interacting_Protein,P_Value
0,Gbm,TP53,TP53_phosphoproteomics_ALPNNTSSS*PQPK_NP_000537.3,True,1.539204e-07
1,Gbm,TP53,TP53BP1_phosphoproteomics_QSQQPMKPIS*PVKDPVS*P...,True,1.351309e-06
2,Gbm,TP53,TP53BP1_phosphoproteomics_S*PEPEVLST*QEDLFDQSN...,True,2.371163e-06
3,Gbm,TP53,TP53BP1_phosphoproteomics_GNLLHFPSS*QGEEEK_NP_...,True,1.266342e-05
4,Gbm,TP53,TP53BP1_phosphoproteomics_SGTAETEPVEQDSS*QPSLP...,True,7.026147e-05
5,Gbm,TP53,TP53BP1_phosphoproteomics_EGDIIPPLTGAT*PPLIGHL...,True,7.413043e-05
6,Gbm,TP53,TP53BP1_phosphoproteomics_DPVS*PAS*QK_NP_00113...,True,9.519195e-05
0,Gbm,TP53,TCEAL3_phosphoproteomics_S65_REDEGEPGDEGQLEDEG...,False,1.559019e-09
1,Gbm,TP53,TCEAL3_phosphoproteomics_S72_QGRS*EGEGKPQGEGK_...,False,3.528867e-08
2,Gbm,TP53,TP53_phosphoproteomics_S315_ALPNNTSSS*PQPK_NP_...,False,1.539204e-07


### Write significant comparisons (if any) to shared CSV file

In [10]:
'''existing_results = pd.read_csv(gene+'_Trans_Results.csv')

updated_results = pd.concat([existing_results, all_significant_comparisons], sort=False)

updated_results.to_csv(path_or_buf = gene + '_Trans_Results.csv', index=False)'''

"existing_results = pd.read_csv(gene+'_Trans_Results.csv')\n\nupdated_results = pd.concat([existing_results, all_significant_comparisons], sort=False)\n\nupdated_results.to_csv(path_or_buf = gene + '_Trans_Results.csv', index=False)"